# Creating Animated tiles from PNGs

## [Tiled web map](https://en.wikipedia.org/wiki/Tiled_web_map)

Most tiled web maps follow certain Google Maps conventions:

- Tiles are 256x256 pixels
- At the outer most zoom level, 0, the entire world can be rendered in a single map tile.
- Each zoom level doubles in both dimensions, so a single tile is replaced by 4 tiles when zooming in. 
- The [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator_projection) projection is used, with latitude limits of around 85 degrees.

OpenStreetMap, Google Maps, MapBox, etc. use the standard known as [Slippy Map Tilenames](https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames) or XYZ:

- Tiles are 256 × 256 pixel PNG files
- Each zoom level (z) is a directory, each column (x) is a subdirectory, and each tile (y) in that column is a file
- Filename (url) format is `http://.../z/x/y.png`

Other standards also exist such as the [Tile Map Service](https://en.wikipedia.org/wiki/Tile_Map_Service) or TMS.

The only difference between XYZ and TMS is a flip in coordinates. TMS has its origin [0,0] at the bottom-left, while the origin for XYZ is located at the top-left.

The conversion between tiles and coordinates has been described [here](https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/) ([source code](https://gist.github.com/maptiler/fddb5ce33ba995d5523de9afdf8ef118#file-globalmaptiles-py)). 

## Table of Contents
## 1. [Setup](#setup)
- **[Python libraries](#libraries)**
- **[Utils](#utils)**
## 2. [Acquire satellite images time series from Earth Engine](#satellite_images)
- **[Get MODIS NDVI collection](#modis)**
- **[Group images by composite date](#composite)**
## 3. [Create Animated tiles from `PNGs`](#tiles_from_pgns)
- **[APNG](#apng_tiles)**
- **[WebM](#webm_tiles)**
- **[WebP](#webp_tiles)**
- **[GIF](#gif_tiles)**
## 4. [Upload tiles to Google cloud storage](#upload_to_gcs)

<a id='setup'></a>
## 1. Setup

<a id='libraries'></a>
### Python libraries

In [46]:
import numpy as np
from dotenv import load_dotenv
from google.cloud import storage
from google.oauth2 import service_account
import glob
import matplotlib.pyplot as plt
import urllib
import requests
from PIL import Image
import ffmpeg
from apng import APNG
import subprocess
import math
import os
import io 
import ee
ee.Initialize()

<a id='utils'></a>
### Utils

In [2]:
class GlobalMercator(object):
    """
    TMS Global Mercator Profile
    ---------------------------
    Functions necessary for generation of tiles in Spherical Mercator projection,
    EPSG:900913 (EPSG:gOOglE, Google Maps Global Mercator), EPSG:3785, OSGEO:41001.
    Such tiles are compatible with Google Maps, Microsoft Virtual Earth, Yahoo Maps,
    UK Ordnance Survey OpenSpace API, ...
    and you can overlay them on top of base maps of those web mapping applications.
    
    Pixel and tile coordinates are in TMS notation (origin [0,0] in bottom-left).
    What coordinate conversions do we need for TMS Global Mercator tiles::
         LatLon      <->       Meters      <->     Pixels    <->       Tile     
     WGS84 coordinates   Spherical Mercator  Pixels in pyramid  Tiles in pyramid
         lat/lon            XY in metres     XY pixels Z zoom      XYZ from TMS 
        EPSG:4326           EPSG:900913                                         
         .----.              ---------               --                TMS      
        /      \     <->     |       |     <->     /----/    <->      Google    
        \      /             |       |           /--------/          QuadTree   
         -----               ---------         /------------/                   
       KML, public         WebMapService         Web Clients      TileMapService
    What is the coordinate extent of Earth in EPSG:900913?
      [-20037508.342789244, -20037508.342789244, 20037508.342789244, 20037508.342789244]
      Constant 20037508.342789244 comes from the circumference of the Earth in meters,
      which is 40 thousand kilometers, the coordinate origin is in the middle of extent.
      In fact you can calculate the constant as: 2 * math.pi * 6378137 / 2.0
      $ echo 180 85 | gdaltransform -s_srs EPSG:4326 -t_srs EPSG:900913
      Polar areas with abs(latitude) bigger then 85.05112878 are clipped off.
    What are zoom level constants (pixels/meter) for pyramid with EPSG:900913?
      whole region is on top of pyramid (zoom=0) covered by 256x256 pixels tile,
      every lower zoom level resolution is always divided by two
      initialResolution = 20037508.342789244 * 2 / 256 = 156543.03392804062
    What is the difference between TMS and Google Maps/QuadTree tile name convention?
      The tile raster itself is the same (equal extent, projection, pixel size),
      there is just different identification of the same raster tile.
      Tiles in TMS are counted from [0,0] in the bottom-left corner, id is XYZ.
      Google placed the origin [0,0] to the top-left corner, reference is XYZ.
      Microsoft is referencing tiles by a QuadTree name, defined on the website:
      http://msdn2.microsoft.com/en-us/library/bb259689.aspx
    The lat/lon coordinates are using WGS84 datum, yeh?
      Yes, all lat/lon we are mentioning should use WGS84 Geodetic Datum.
      Well, the web clients like Google Maps are projecting those coordinates by
      Spherical Mercator, so in fact lat/lon coordinates on sphere are treated as if
      the were on the WGS84 ellipsoid.
     
      From MSDN documentation:
      To simplify the calculations, we use the spherical form of projection, not
      the ellipsoidal form. Since the projection is used only for map display,
      and not for displaying numeric coordinates, we don't need the extra precision
      of an ellipsoidal projection. The spherical projection causes approximately
      0.33 percent scale distortion in the Y direction, which is not visually noticable.
    How do I create a raster in EPSG:900913 and convert coordinates with PROJ.4?
      You can use standard GIS tools like gdalwarp, cs2cs or gdaltransform.
      All of the tools supports -t_srs 'epsg:900913'.
      For other GIS programs check the exact definition of the projection:
      More info at http://spatialreference.org/ref/user/google-projection/
      The same projection is degined as EPSG:3785. WKT definition is in the official
      EPSG database.
      Proj4 Text:
        +proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0
        +k=1.0 +units=m +nadgrids=@null +no_defs
      Human readable WKT format of EPGS:900913:
         PROJCS["Google Maps Global Mercator",
             GEOGCS["WGS 84",
                 DATUM["WGS_1984",
                     SPHEROID["WGS 84",6378137,298.2572235630016,
                         AUTHORITY["EPSG","7030"]],
                     AUTHORITY["EPSG","6326"]],
                 PRIMEM["Greenwich",0],
                 UNIT["degree",0.0174532925199433],
                 AUTHORITY["EPSG","4326"]],
             PROJECTION["Mercator_1SP"],
             PARAMETER["central_meridian",0],
             PARAMETER["scale_factor",1],
             PARAMETER["false_easting",0],
             PARAMETER["false_northing",0],
             UNIT["metre",1,
                 AUTHORITY["EPSG","9001"]]]
    """
    
    def __init__(self, tileSize=256):
        "Initialize the TMS Global Mercator pyramid"
        self.tileSize = tileSize
        self.initialResolution = 2 * math.pi * 6378137 / self.tileSize
        # 156543.03392804062 for tileSize 256 pixels
        self.originShift = 2 * math.pi * 6378137 / 2.0
        # 20037508.342789244
        
    def MetersToLatLon(self, mx, my ):
        "Converts XY point from Spherical Mercator EPSG:900913 to lat/lon in WGS84 Datum"

        lon = (mx / self.originShift) * 180.0
        lat = (my / self.originShift) * 180.0

        lat = 180 / math.pi * (2 * math.atan( math.exp( lat * math.pi / 180.0)) - math.pi / 2.0)
        return lat, lon

    def Resolution(self, zoom ):
        "Resolution (meters/pixel) for given zoom level (measured at Equator)"
        
        # return (2 * math.pi * 6378137) / (self.tileSize * 2**zoom)
        return self.initialResolution / (2**zoom)
        
    def PixelsToMeters(self, px, py, zoom):
        "Converts pixel coordinates in given zoom level of pyramid to EPSG:900913"

        res = self.Resolution( zoom )
        mx = px * res - self.originShift
        my = py * res - self.originShift
        return mx, my
        

    def TileBounds(self, tx, ty, zoom):
        "Returns bounds of the given tile in EPSG:900913 coordinates"
        
        minx, miny = self.PixelsToMeters( tx*self.tileSize, ty*self.tileSize, zoom )
        maxx, maxy = self.PixelsToMeters( (tx+1)*self.tileSize, (ty+1)*self.tileSize, zoom )
        return ( minx, miny, maxx, maxy )
        
    def TileLonLatBounds(self, tx, ty, zoom ):
        "Returns bounds of the given tile in latutude/longitude using WGS84 datum"

        bounds = self.TileBounds( tx, ty, zoom)
        minLat, minLon = self.MetersToLatLon(bounds[0], bounds[1])
        maxLat, maxLon = self.MetersToLatLon(bounds[2], bounds[3])
         
        return ( minLon, minLat, maxLon, maxLat )
    
    def GoogleTile(self, tx, ty, zoom):
        "Converts TMS tile coordinates to Google Tile coordinates and vice versa"
        
        # coordinate origin is moved from bottom-left to top-left corner of the extent
        return tx, (2**zoom - 1) - ty

**create_movie_tiles_from_pngs**

In [3]:
def create_movie_from_pngs(input_files, output_file, output_format='apng', framerate=4):
    
    if output_format == 'apng':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate, pix_fmt='rgba')
            .output(output_file, **{'plays': 0})
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        #ffmpeg -framerate 3 -i ./data/movie/movie_%03d.png -plays 0 ./data/movie/movie.apng
        
    if output_format == 'webm':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate, pix_fmt='yuva420p')
            .output(output_file, **{'auto-alt-ref': 0})
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        # ffmpeg -framerate 4 -start_number 000 -i ./data/movie/movie_%03d.png -vf scale=490:512 -c:v libvpx -pix_fmt yuva420p -auto-alt-ref 0 -metadata:s:v:0 alpha_mode="1" ./data/movie/movie.webm
       
    if output_format == 'webm':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate, pix_fmt='yuva420p')
            .output(output_file, **{'auto-alt-ref': 0})
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        # ffmpeg -framerate 4 -start_number 000 -i ./data/movie/movie_%03d.png -vf scale=490:512 -c:v libvpx -pix_fmt yuva420p -auto-alt-ref 0 -metadata:s:v:0 alpha_mode="1" ./data/movie/movie.webm
        
    if output_format == 'webp':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate)
            .output(output_file, vcodec='libwebp')
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        # ffmpeg -framerate 4 -i ./data/movie/movie_%03d.png -c:v libwebp ./data/movie/movie.webp
        
    if output_format == 'gif':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate)
            .output(output_file, vcodec='gif')
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        # ffmpeg -f image2 -framerate 1 -i ./data/movie/movie_%03d.png  ./data/movie/movie.gif
        
    if output_format == 'mp4':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate, pix_fmt='yuv420p')
            .output(output_file, vcodec='libx264', crf=20)
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        # ffmpeg -framerate 4 -start_number 000 -i ./data/movie/movie_%03d.png -vf scale=490:512 -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p movie.mp4

    if output_format == 'mov':
        out, _ = (
            ffmpeg
            .input(input_files, framerate=framerate, pix_fmt='rgba')
            .output(output_file, vcodec='png')
            .run(capture_stdout=True, overwrite_output=True)
        )
        # Corresponding command line code
        # ffmpeg -framerate 4 -start_number 000 -i ./data/movie/movie_%03d.png -vf scale=490:512 -vcodec png -pix_fmt rgba ./data/movie/movie.mov

**compress_img**

In [4]:
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"

In [5]:
def compress_img(img_path):
    # load the image to memory
    img = Image.open(img_path)
    # get the original image size in bytes
    image_size = os.path.getsize(img_path)
    # print the size before compression
    print("Size before compression:", get_size_format(image_size))
    # compress the image
    #img = img.convert("P", palette=Image.ADAPTIVE, colors=256)
    img = img.convert("P", palette=Image.WEB, colors=256)
    # save the image with the optimize set to True
    img.save(img_path, optimize=True)

    # get the new image size in bytes
    new_image_size = os.path.getsize(img_path)
    # print the new size in a good format
    print("Size after compression:", get_size_format(new_image_size))
    # calculate the saving bytes
    saving_diff = new_image_size - image_size
    # print the saving percentage
    print(f"Image size change: {saving_diff/image_size*100:.2f}% of the original image size.")

**compress_png**

In [66]:
def compress_png(img_path):
    """
    Compress PNGs with pngquant: a command-line utility and a library for lossy compression of PNG images.
    The conversion reduces file sizes significantly (often as much as 70%) and preserves full alpha transparency. 
    Generated images are compatible with all web browsers and operating systems.
    """
    cmd = f"pngquant --ext _new.png --quality=65-80 {img_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
        # Remove PNGs
        os.remove(img_path)
        # Rename PNGs
        os.rename(img_path.replace(".png", "_new.png"), img_path)
    else:
        print("Task failed")
    print("Finished processing")
    

**create_animated_tiles_from_pngs**

In [71]:
GM = GlobalMercator()

def TileCoordinates (z):
    """Tile coordinates for a given z"""
    return np.arange(int(np.sqrt(4**z)))

def create_animated_tiles_from_pngs(path, dimensions, minZoom, maxZoom, output_format='apng', compress=True):    
    ## Get bounding box
    zVals = np.arange(minZoom,maxZoom+1)
    for z in zVals:
        xVals = TileCoordinates(z)
        yVals = TileCoordinates(z)
        for x in xVals:
            for y in yVals:
                print(f'Creating: '+str(z)+'/'+str(x)+'/'+f'{str(y)}.png')
                # Get bounding box
                x_TMS, y_TMS = GM.GoogleTile(x,y,z)
                bbox = list(GM.TileLonLatBounds(x_TMS,y_TMS,z))
        
                # Replace 180.0 with 179.99
                bbox = [x if x != 180.0 else 179.99 for x in bbox]
                bbox = [x if x != -180.0 else -179.99 for x in bbox]

                ## Create a group of images  
                visSave = {'dimensions': dimensions, 'format': 'png', 'crs': 'EPSG:3857', 'region':bbox} 

                if not os.path.exists(path+str(z)+'/'+str(x)+'/'+'tmp/'):
                    os.makedirs(path+str(z)+'/'+str(x)+'/'+'tmp/')
                
                for n, image in enumerate(images):
                    number = '{:03d}'.format(n)
                    image = ee.Image(image)

                    url = image.getThumbURL(visSave)

                    if os.path.exists(path+str(z)+'/'+str(x)+'/'+'tmp/'+f'{number}.png'):
                        os.remove(path+str(z)+'/'+str(x)+'/'+'tmp/'+f'{number}.png')

                    urllib.request.urlretrieve(url, path+str(z)+'/'+str(x)+'/'+'tmp/'+f'{number}.png')

                    # Compress PNGs with pngquant
                    if compress:
                        #compress_img(path+str(z)+'/'+str(x)+'/'+'tmp/'+f'{number}.png')
                        compress_png(path+str(z)+'/'+str(x)+'/'+'tmp/'+f'{number}.png')

                ## Create tiles from a list of PNGs.
                png_files = [f for f in os.listdir(path+str(z)+'/'+str(x)+'/'+'tmp/') if f.endswith('.png')]
                png_files = sorted(png_files, key=lambda x: float(x.split('.')[0]))
                png_files = [path+str(z)+'/'+str(x)+'/'+'tmp/' + i for i in png_files]
                
                #APNG.from_files(png_files, delay=1).save(path+'/'+str(z)+'/'+str(x)+'/'+str(y)+'.png')
                create_movie_from_pngs(path+str(z)+'/'+str(x)+'/'+'tmp/'+'%03d.png', path+'/'+str(z)+'/'+str(x)+'/'+str(y)+f'.{output_format}', output_format)

                # Remove PNGs
                [os.remove(file) for file in png_files]

            # Remove tmp folder
            os.removedirs(path+str(z)+'/'+str(x)+'/'+'tmp/')

**upload_local_directory_to_gcs**

In [11]:
def upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path):
    """Uploads a directory to the bucket."""
    load_dotenv()
    credentials_dict = {
          "type": "service_account",
          "project_id": os.getenv("PROJECT_ID"),
         "private_key_id": os.getenv("PRIVATE_KEY_ID"),
          "private_key":os.getenv("PRIVATE_KEY"),
         "client_email": os.getenv("CLIENT_EMAIL"),
          "token_uri":os.getenv("TOKEN_URI"),   
    } 
    

    credentials = service_account.Credentials.from_service_account_info(credentials_dict)
    storage_client = storage.Client(project='project_id', credentials=credentials)
    
    bucket = storage_client.bucket(bucket_name)
    rel_paths = glob.glob(local_path + '/**', recursive=True)

    for local_file in rel_paths:
        remote_path = f'{destination_blob_path}{"/".join(local_file.split(os.sep)[len(local_path.split(os.sep))-1:])}'
        if os.path.isfile(local_file):
            blob = bucket.blob(remote_path)
            print(
                "File {} uploaded to {}.".format(
                    local_file, remote_path
                )
            )
            blob.upload_from_filename(local_file)

<a id='satellite_images'></a>
## 2. Acquire satellite images time series from Earth Engine

<a id='modis'></a>
**Get MODIS NDVI collection**

Retrieve the [MODIS Terra Vegetation Indices 16-Day Global 1km dataset](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2) as an ee.ImageCollection and select the NDVI band.

In [12]:
col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI').filterDate('2013-01-01', '2014-01-01')

<a id='composite'></a>
**Group images by composite date**

In [13]:
# Get liost of dates
def transform_dates(img):
    return ee.Feature(None, {'date': img.date().format('YYYY-MM-dd')})

dates = col.map(transform_dates).distinct('date').aggregate_array('date').getInfo()

# Create a group of images
def visFun(img, vis):
    """
    Define a function to convert an image to an RGB visualization image and copy
    properties from the original image to the RGB image.
    """
    return img.visualize(**vis).copyProperties(img, img.propertyNames())

visParams = {'min': 0.0, 'max': 9000.0,
             'palette': [
                'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                '012E01', '011D01', '011301'],
}

images = []
for n in range(len(dates)-1): 
    image = col.filterDate(dates[n], dates[n+1]).first()
    images.append(visFun(image, visParams))

<a id='tiles_from_pgns'></a>
## 3. Create Animated tiles from `PNGs`

In [27]:
minZoom = 0
maxZoom = 0
dimensions=256

<a id='apng_tiles'></a>
**APNG tiles**

In [36]:
path = '../../datasets/processed/movie/tiles/APNGs/'

In [72]:
create_animated_tiles_from_pngs(path, dimensions, minZoom, maxZoom, output_format='apng', compress=True)

Creating: 0/0/0.png
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/000.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/001.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/002.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/003.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/004.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/005.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/006.png
Task created
Finished processing
Processing: pngquant --ext _new.png --quality=65-80 ./data/movie/tiles/APNGs/0/0/tmp/007.png
Task 

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

Error: ffmpeg error (see stderr output for detail)

<a id='webm_tiles'></a>
**WebM tiles**

In [9]:
path = '../../datasets/processed/movie/tiles/WebMs/'

In [ ]:
create_animated_tiles_from_pngs(path, dimensions, minZoom, maxZoom, output_format='webm')

<a id='webp_tiles'></a>
**WebP tiles**

In [19]:
path = '../../datasets/processed/movie/tiles/WebPs/'

In [ ]:
create_animated_tiles_from_pngs(path, dimensions, minZoom, maxZoom, output_format='webp')

<a id='gif_tiles'></a>
**GIF tiles**

In [14]:
path = '../../datasets/processed/movie/tiles/GIFs/'

In [ ]:
create_animated_tiles_from_pngs(path, dimensions, minZoom, maxZoom, output_format='gif')

<a id='upload_to_gcs'></a>
## 4. Upload tiles to Google cloud storage

In [9]:
# Upload `APNG` files to GCS
bucket_name = 'skydipper_materials'
local_path = '../../datasets/processed/movie/tiles/GIFs/'
destination_blob_path = 'movie-tiles/MODIS/GIFs/'

upload_local_directory_to_gcs(bucket_name, local_path, destination_blob_path)

File ./data/movie/tiles/GIFs/3/5/7.gif uploaded to movie-tiles/MODIS/GIFs/3/5/7.gif.
File ./data/movie/tiles/GIFs/3/5/3.gif uploaded to movie-tiles/MODIS/GIFs/3/5/3.gif.
File ./data/movie/tiles/GIFs/3/5/2.gif uploaded to movie-tiles/MODIS/GIFs/3/5/2.gif.
File ./data/movie/tiles/GIFs/3/5/0.gif uploaded to movie-tiles/MODIS/GIFs/3/5/0.gif.
File ./data/movie/tiles/GIFs/3/5/1.gif uploaded to movie-tiles/MODIS/GIFs/3/5/1.gif.
File ./data/movie/tiles/GIFs/3/5/5.gif uploaded to movie-tiles/MODIS/GIFs/3/5/5.gif.
File ./data/movie/tiles/GIFs/3/5/4.gif uploaded to movie-tiles/MODIS/GIFs/3/5/4.gif.
File ./data/movie/tiles/GIFs/3/5/6.gif uploaded to movie-tiles/MODIS/GIFs/3/5/6.gif.
File ./data/movie/tiles/GIFs/3/3/7.gif uploaded to movie-tiles/MODIS/GIFs/3/3/7.gif.
File ./data/movie/tiles/GIFs/3/3/3.gif uploaded to movie-tiles/MODIS/GIFs/3/3/3.gif.
File ./data/movie/tiles/GIFs/3/3/2.gif uploaded to movie-tiles/MODIS/GIFs/3/3/2.gif.
File ./data/movie/tiles/GIFs/3/3/0.gif uploaded to movie-tiles/MO